In [27]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
# path = r"x264/sampledConfigurations_distBased_t3.csv"
#path = r"sampledConfigurations_distBased_t3.csv"
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

# for config_name in samples_config:
#     path = r"sampledConfigurations_"+config_name+"_t3.csv"
# #     df.append(pd.read_csv(path, sep=';'))
#     display(path)

# df = pd.read_csv(path, sep=';')


In [2]:
# df_importance = pd.read_csv("feature_importanceRF.csv")
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
# df_importance.head()

In [28]:
#size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
#              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
#?              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

size_methods = ["Performance"]
#additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]

In [29]:
def features_engine(df):
    
    engine_values = []
    for i, j in df.iterrows():
        num1s = 0
        num1s = j.value_counts()[1]
        engine_values.append(num1s)
    
    df['#yes'] = engine_values
    

In [30]:
path = r"sampledConfigurations_distBased_t3.csv"
df = pd.read_csv(path, sep=';')
features_engine(df)
display(df)

,root,encryption,compression,compressionBzip2,compressionGzip,compressionLzo,compressionZpaq,compressionLrzip,level,level1,...,level4,level5,level6,level7,level8,level9,unlimitedWindowSize,disableCompressibilityTesting,Performance,construction
0,1,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,208900,5
1,1,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,210450,5
2,1,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,283750,5
3,1,0,1,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,339780,5
4,1,0,1,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,340590,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,1,1,1,5765570,8
246,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,55520,6
247,1,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,1,57860,6
248,1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,1,58700,6


In [31]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [32]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [33]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
#df_result  = pd.DataFrame(columns = ['algorithm_name', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [34]:
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

ft_importance_enable = True
# def runFeatureSelection(df, config_name, num_features): 
for config_name in samples_config:
    for samplePerc in [0.3,0.4,0.5,0.6]:
        for t in [1,2,3]:
            
            path_name = config_name+"_t"+str(t)
            path = r"sampledConfigurations_"+path_name+".csv"
            
            df = pd.read_csv(path, sep=';')
            num_features = round(samplePerc*len(df.columns))

            df_importance = pd.read_csv("feature_importance_"+path_name+".csv")
            df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
            
            print("DF with Feature Importance")
            print("Sample features number: "+ str(num_features))
            display(df_importance)
            df_importance.head()

            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}

            for ft_dummyfication in [False, True]:
                if ft_dummyfication:
                    #Creating dummy variables in pandas
                    df = pd.get_dummies(df, columns=list(df.drop(columns=size_methods).columns.values))
#                     df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=size_methods).columns.values))
                    df_importance = pd.get_dummies(df_importance, columns=list(df_importance.columns.values))

                if ft_importance_enable:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                'RandomForest': df_importance.columns[:num_features],}
                    
                else:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                     'RandomForest': df.columns[:num_features],}

                for origin_ft_selection in ft_selection:
                    for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:

                        x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
                        x_train, y_train = np.array(x_train.values), np.array(y_train.values)
                        
                        print(x_train)
                        print(y_train)
                        
                        if len(x_train) > 3:

                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression()}
                        else:
                            if len(x_train) >= 2:
                                model = {'LinearRegression': LinearRegression(),
                                         'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                         'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                         'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                         'PolynomialRegression': LinearRegression()}
                        for key in model:
                            if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                                print("It doesn't scale")
                            else:
                                if (key == 'PolynomialRegression'):
                                    #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                    x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                    x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                                res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)

                                if (key == 'PolynomialRegression'):
                                    model[key].coef_ = model[key].coef_[:num_features]

#                                 print(model[key].coef_)
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                                df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
            display(df_result)
            if ft_importance_enable:
                df_result.to_csv("results_with_feature_importance_"+path_name+"_"+str(num_features)+".csv", header=True)
            else:
                df_result.to_csv("results_"+path_name+"_"+str(num_features)+".csv", header=True)

DF with Feature Importance
Sample features number: 6


,compressionLrzip,compressionBzip2,encryption,level1,compressionGzip,compression
0,0,1,0,1,0,1
1,0,0,0,1,1,1
2,0,0,0,0,1,1
3,0,0,0,1,0,0
4,0,0,0,0,0,0
5,0,1,1,0,0,1
6,0,1,1,0,0,1
7,0,1,1,0,0,1
8,0,0,1,0,0,0
9,0,0,1,0,0,0


[[1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1]
 [1 1 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0]
 [1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]
 [1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1]
 [1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0]
 [1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0]
 [1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
 [1 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]]
[ 61280 263150  41150  62270  60850  60850 832070  80360  56430  61880
  43560  64550 274670  59650  61680 271090]
It doesn't scale
[[1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1]
 [1 1 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1]
 [1 1 1

It doesn't scale
[[1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0
  1]
 [1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0
  1]
 [1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1
  0]
 [1 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
  0]
 [1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1
  0]
 [1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0
  1]
 [1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1
  0]
 [1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 1 1
  0]
 [1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1
  0]
 [1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1
  0]
 [1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0
  1]
 [1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 1 0 1 0 1
  0]]
[ 60850  60850 832070  80360  5643

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.1,compressionLrzip 4.667898e+05 ...,count 2.000000 mean 10.434399 std ...
1,Lasso,False,None,"Index(['root', 'encryption', 'compression', 'c...",Lasso(alpha=25),Performance,0.1,compressionLrzip 674369.793373 compressionB...,count 2.000000 mean 12.152904 std ...
2,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(alpha=4),Performance,0.1,compressionLrzip 112621.112815...,count 2.000000 mean 42.678642 std ...
3,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=25, l1_ratio=1.0)",Performance,0.1,compressionLrzip 674369.793373 compressionB...,count 2.000000 mean 12.152904 std ...
4,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.2,compressionLrzip 4.669864e+05 ...,count 4.000000 mean 9.067887 std ...
...,...,...,...,...,...,...,...,...,...
1064,LinearRegression,True,RandomForest,"Index(['compressionLrzip_0', 'compressionLrzip...",LinearRegression(),Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 280.941068 std ...
1065,Lasso,True,RandomForest,"Index(['compressionLrzip_0', 'compressionLrzip...","Lasso(alpha=25, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 280.941068 std ...
1066,Ridge,True,RandomForest,"Index(['compressionLrzip_0', 'compressionLrzip...",Ridge(alpha=25),Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 280.941068 std ...
1067,ElasticNet,True,RandomForest,"Index(['compressionLrzip_0', 'compressionLrzip...","ElasticNet(alpha=25, l1_ratio=0.0, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 280.941068 std ...


...

It doesn't scale
[[1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0]
 [1 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 1]
 [1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1]
 [1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0]
 [1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0 1]
 [1 0 1 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 1 0]
 [1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 0 1]
 [1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1]
 [1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0]]
[ 53870  45770 262750 686140 274670  56570 220350  41900  92800  59650]
It doesn't scale
[[1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0]
 [1 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0 0 1]
 [1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.1,compressionLrzip 4.667898e+05 ...,count 2.000000 mean 10.434399 std ...
1,Lasso,False,None,"Index(['root', 'encryption', 'compression', 'c...",Lasso(alpha=25),Performance,0.1,compressionLrzip 674369.793373 compressionB...,count 2.000000 mean 12.152904 std ...
2,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(alpha=4),Performance,0.1,compressionLrzip 112621.112815...,count 2.000000 mean 42.678642 std ...
3,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=25, l1_ratio=1.0)",Performance,0.1,compressionLrzip 674369.793373 compressionB...,count 2.000000 mean 12.152904 std ...
4,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.2,compressionLrzip 4.669864e+05 ...,count 4.000000 mean 9.067887 std ...
...,...,...,...,...,...,...,...,...,...
3008,LinearRegression,True,RandomForest,"Index(['compressionZpaq_0', 'compressionZpaq_1...",LinearRegression(),Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 42.494087 std ...
3009,Lasso,True,RandomForest,"Index(['compressionZpaq_0', 'compressionZpaq_1...","Lasso(alpha=25, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 42.494087 std ...
3010,Ridge,True,RandomForest,"Index(['compressionZpaq_0', 'compressionZpaq_1...",Ridge(alpha=25),Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 42.494087 std ...
3011,ElasticNet,True,RandomForest,"Index(['compressionZpaq_0', 'compressionZpaq_1...","ElasticNet(alpha=25, l1_ratio=0.0, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 17.000000 mean 42.494087 std ...


DF with Feature Importance
Sample features number: 6


,compressionZpaq,level9,level8,level3,compressionLrzip,unlimitedWindowSize
0,0,0,0,0,1,0
1,0,1,0,0,1,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,1,0,0
...,...,...,...,...,...,...
94,1,0,0,1,0,1
95,1,0,0,0,0,1
96,1,0,0,0,0,1
97,1,0,0,0,0,1


[[1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 1]
 [1 0 0 ... 0 1 0]
 ...
 [1 0 1 ... 1 0 0]
 [1 1 1 ... 0 1 1]
 [1 1 1 ... 0 1 0]]
[  64550   56570   43330 1070520   42910  223040   46950   51500   89350
 5615690  220350  224880   44110   41150   56430   53200   87690  218000
   48050   41900  686140  198500  222830   92800  234870   41830   45270
   54820   41790 2272580   56980  263430   59380  833620   43560 2270490
  395130 2282210 2267650  295950  213220   42300   61280   80360   80780
  248940  678540   53500  200050   59060 5588490   52650   41260   64550
   42230   47130   43660  145660 2280880   51150  212880  209160   45980
  832650  212530   53870   49760   64270  338900   40430  350470   56570
  274670  827290 2278560   44370  208350   45770  278140  155300 5619060
  234680  216030   45720  262750  683560 5759100   62270  681430]


In [ ]:
len(x_train)

In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result['accuracy'])):
    rs.append(df_result['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()